In [1]:
# Import
import numpy as np
import matplotlib.pyplot as plt
from Parametrs import *
from Structure import *

import jsonpickle as jp
import shutil
import re
import inspect
from tqdm import tqdm


def clear_savedir(path_save):
    """Clear directory in VTK folder"""

    try:
        shutil.rmtree(path_save)
    except:
        pass


def view_file1D(
    name,
    dir='/home/VTK/System1D_files/',
    view_dir='/home/VTK/System1D_files/0',
):
    shutil.copytree(dir + name, view_dir, dirs_exist_ok=True)


def repr_str(func, rules: list = {}) -> str:
    # TODO: to module
    # FIXME: (( to (\n(
    rep = str(func)
    rep = re.sub(r'\{ A \| A_\{i_(\d|\{\d+\})\} \=', '', rep)
    rep = re.sub(r'\[i_(\d|{\d+})]', '', rep)
    rep = re.sub(r'dx.+\)', 'dx', rep)
    rep = re.sub(r'ds.+\)', 'ds', rep)
    for key, raw in rules.items():
        rep = rep.replace(str(raw),key)

    base = {
        'v_0[0]': 'v',
        'v_0[1]': 'u',
        '-1 * ': '-',
        '+ -': '-',
        'f[0]': 'N',
        'f[1]': 'P',
        '{': '{\n',
        '} *': '\n} *',
    }
    for key, raw in base.items():
        rep = rep.replace(key, raw)

    beauty = {
        '(grad(P))': 'grad(P)',
        '(grad(N))': 'grad(N)',
        '(grad(u))': 'grad(u)',
        '(grad(v))': 'grad(v)',
        '})': ')',
        '+ (': '\n   +(',
    }
    for key, raw in beauty.items():
        rep = rep.replace(key, raw)
    return rep

In [2]:
# # For linux
# p_save = Param_save(dir_save=r'D:\WORKS\PolimerSolve\Solver1D\Solves\1\system1D_anotaton.txt')
# p_save.dir_save
# path= p_save.dir_save
# path += array[0]
# path +=p_save.file_name+'_anotation.txt'
# path

In [6]:
import os
l =[]
for  i in os.walk('Solves'):
    l.append(i[0])
l = l[1:]
_array = [l[i].split('\\')[1] for i in range(len(l))]
_array

['1', '2', '3', '4', '6']

In [7]:
array = tqdm(desc=f'Quering...', iterable=_array)

for name in array:
    path = r'D:\WORKS\PolimerSolve\Solver1D\Solves\file\system1D_anotaton.txt'.replace(
        'file', name
    )
    with open(file=path, mode='r') as file:
        dump = jp.decode(file.read())

    solve_confs = {
        key: str(value)
        for key, value in dump.solve_confs.__dict__.items()
    }
    with database_connection:
        note = {}
        note.update({'mesh': Mesh1D.get_or_create(**dump.mesh.__dict__)})
        note.update({'light': Light1D.get_or_create(**dump.light.__dict__)})
        note.update({'solver': Solver.get_or_create(**solve_confs)})
        note.update({
            'task':
            Task.get_or_create(
                name=name,
                desc=dump.save.desc,
                T=dump.time.T,
                dt=dump.time.dt,
                mesh=note['mesh'][0],
                N0=dump.initial.N0,
                P0=dump.initial.P0,
                light=note['light'][0],
                solver=note['solver'][0]
            )
        })
        for key, value in dump.consts.__dict__.items():
            Const.get_or_create(parametr=key, value=value, task=note['task'][0])

Quering...: 100%|██████████| 5/5 [00:10<00:00,  2.08s/it]
